In [1]:
import scrapy
import requests
from bs4 import BeautifulSoup
import datetime
from textblob import TextBlob
import pandas as pd
import json
import numpy as np
import re

In [4]:
def get_date(soup):
    """ 
        Extracting the date of publishing / updating of the article
    """
    # Getting date of a video published on the article because it's the only date i found that can fit
    if soup.find('meta', property='article:modified_time'):
        date = soup.find('meta', property='article:modified_time').get('content')
        art_published_datetime = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S%z').date()
    else:
        art_published_datetime = datetime.date.today()
    return art_published_datetime

In [5]:
def get_lang(soup):
    
    """ 
        Extracting the language the article is written in
    """
    
    if soup.find('meta', property='og:locale'):
        art_lang = soup.find('meta', property='og:locale').get('content')
    elif soup.find('html').get('lang'):
        art_lang = soup.find('html').get('lang')
    else:
        art_lang = 'fr-FR'
    return art_lang

In [6]:
def get_title(soup):
    """ 
        Extracting the title of the article
    """
    if soup.find('meta', property = "og:title"):
        art_title = soup.find('meta', property = "og:title").get('content')
    elif soup.find('title'):
        art_title = soup.find('title').get_text()
    else :
        art_title = "no_data"
        #à voir avec des expressions regulières pour recup dans le lien du site la partie qui nous interesse
        #for val in re.finditer("/+(\w)+/+(\w)+", url): 
            #return val.group(0)
    return art_title

In [7]:
def get_url(soup, url):
    """ 
        Extracting url of the article
    """
    if soup.find('meta', property = "og:url"):
        art_url = soup.find('meta', property = "og:url").get('content')
    elif soup.find('link', rel = 'canonical'):
        art_url = soup.find('link', rel = 'canonical').get('href')
    else :  #Si jamais on trouve pas, 2e paramètre url et on l'assigne (LUL)
        art_url = url
    return art_url

In [8]:
def get_source_url(soup):
    """ 
        Article's source url
    """
    for val in re.finditer("(\w)+://[^/]+/", url):
        return val.group(0)

In [9]:
def get_source_name(soup):
    """ 
        Extracting article's source name
    """
    src_name = getSourceUrl(soup).replace('https://www.','').replace('.fr/','')
    
    return src_name

In [10]:
def get_source_type(soup):
    """ 
        Extracting article's source type
    """
    src_type = "xpath_source"
    
    return src_type

In [11]:
def get_img(soup):
    """ 
        Extracting article's image
    """
    if soup.find('meta', property = "og:image"):
        src_img = soup.find('meta', property = "og:image").get('content')
    else :
        src_img = "no_data"
    return src_img

In [12]:
def get_art_auth(soup):
    """ 
        Extracting article's author
    """
    if soup.find('meta', attrs={'name':"twitter:data1"}) :
        art_auth = soup.find('meta', attrs={'name':"twitter:data1"}).get('content')
    elif soup.find('div', class_ = "td-post-author-name") :
        art_auth = find('div', class_ = "td-post-author-name").text
    elif soup.find("a", {"class": "nomAuteur"}):
        art_auth = soup.find("a", {"class": "nomAuteur"}).get('content')
    else :
        art_auth = soup.find_all(string = re.compile("écrit par (\w)+ (\w)+"))
        #'no_data'
    
    return art_auth

In [13]:
def get_tag(soup):
    ##keywords or tags
    """
        Extracting article's tags if they exist
    """
    if soup.find('meta', attrs={'name':"keywords"}):
        art_tag = soup.find('meta', attrs={'name':"keywords"}).get('content')
    else:
        art_tag = soup.find_all(string=['tag', 'tags', 'keywords']) ##marche pas trop trop
    return art_tag

In [129]:
def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span']
    if soup.find('article') is not None:
        content_html = soup.find('article')
        ### add if soup.find('article').find_all(list_balises) is not None:
        content = soup.find('article').find_all(list_balises, string = True)
        content = ' '.join(tag.text for tag in content).replace('\xa0', '').strip()
### Second way : get only the text in the div tag(s) (else member)
#         list_p = soup.find('article').find_all('p', string = True)
#         list_parent_p = [p.parent for p in list_p]
#         if len(set(list_parent_p)) == 1:
#             content_html = list_parent_p[0]
#             content = ' '.join(
#                 tag.text for tag in content_html.children if tag.name in list_balises).replace('\xa0', '').strip()
#         else:
#             content_html = soup.find('article')
#             content = [el.get_text() for el in list_parent_p]
#             content = ' '.join(content).replace('\xa0', '').strip()
    else:
        list_div = list()
        for el in soup.find_all('div'):
            if el.find_all('p', recursive = False):
                list_div.append(el)
        index_max = np.argmax([len(block.find_all('p')) for block in list_div])
        content_html = list_div[index_max]
        content = ' '.join(
            tag.text for tag in content_html.children if tag.name in list_balises).replace('\xa0', '').strip()
    return content_html, content

In [130]:
URLs = ['https://www.fnccr.asso.fr/article/big-data-territorial-publication-de-letude-de-la-fnccr/',
        'https://www.banquedesterritoires.fr/big-data-territorial-la-gestion-des-donnees-un-enjeu-davenir-pour-les-collectivites', 
        'https://www.theinnovation.eu/comment-tuer-linnovation-avec-lanalyse-financiere/45',
        'https://www.lemondeinformatique.fr/actualites/lire-les-salaries-et-les-dirigeants-percoivent-differement-la-transformation-digitale-81352.html',
        'https://www.cnil.fr/fr/les-collectivites-territoriales-et-lopen-data-concilier-ouverture-des-donnees-et-protection-des',
        'https://www.inserm.fr/actualites-et-evenements/actualites/ondes-electromagnetiques-faut-il-craindre-5g',
        'https://www.parlonsrh.com/raisons-utiliser-lintelligence-artificielle-dans-gestion-gpec/',
        'https://www.myrhline.com/actualite-rh/de-la-gpec-et-au-workforce-planning-les-5-evolutions-a-connaitre.html',
        'https://grh-multi.net/fr/2016/05/compte-rendu-de-levenement-big-data-gpec/',
        'https://changethework.com/gestion-paie-innovations/',
        'https://changethework.com/chatbot-rh-recrutement/',
        'http://sabbar.fr/management/le-management-strategique-et-le-management-operationnel/#:~:text=Le%20management%20op%C3%A9rationnel%20correspond%20aux,pour%20atteindre%20les%20objectifs%20fix%C3%A9s.',
        # 'http://uis.unesco.org/fr/topic/donnees-sur-linnovation',
        'http://www.linternaute.com/ville/classement/villes/population',
        'http://www.territorial.fr/PAR_TPL_IDENTIFIANT/722/TPL_CODE/TPL_OUVR_NUM_FICHE/PAG_TITLE/La+gestion+en+AP-CP+et+la+programmation+des+investissements/53-dossiers-d-expert.htm',
        'https://apnews.com/',
        'https://blockchainfrance.net/decouvrir-la-blockchain/c-est-quoi-la-blockchain/',
        'https://citoyen-ne-s-de-marseille.fr/cest-quoi-les-autorisations-de-programmes/',
        'https://dataanalyticspost.com/Lexique/algorithmes-genetiques/']

In [131]:
url = URLs[1]

In [132]:
print(url)
get_content(url)[1]

https://www.banquedesterritoires.fr/big-data-territorial-la-gestion-des-donnees-un-enjeu-davenir-pour-les-collectivites


'Données Publiques -  Publié le Cohésion des territoires,\u2002 Fonction publique,\u2002 Organisation territoriale, élus et institutions Launch Print Window Share this Page \n      Plusieurs études récentes viennent éclairer les enjeux de la gouvernance des données à l\'échelle locale. La dernière en date émane de la FNCCR, en partenariat avec la Caisse des Dépôts. Si la loi pour une République numérique a permis de mettre le sujet des données à l\'agenda des collectivités, un long chemin reste à faire pour tirer parti du big data dans le cadre de l\'action publique et pour coordonner les initiatives des différents acteurs publics et privés.\n     La Fédération nationale des collectivités concédantes et régies (FNCCR) a rendu publique le 17 novembre une étude, co-pilotée par la Caisse des Dépôts, se penchant sur l\'enjeu émergent du "big data territorial". Les activités des entités publiques et de leurs opérateurs génèrent en effet de plus en plus de données, qui pourraient être d\'une

In [100]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
all_div = soup.find_all('div')
print(all_div[0].prettify())
for i in all_div[0].children:
    print(i)

<div class="skip-links position-fixed">
 <ul class="list-inline m-0 p-0">
  <li class="list-inline-item">
   <a class="skip-links__link visually-hidden focusable" data-a11y-dialog-show="modal-dialog-access-config" data-target="#a11yModal" data-toggle="modal" href="#modal_access" role="button">
    Open accessibility window
   </a>
  </li>
  <li class="list-inline-item">
   <a class="skip-links__link visually-hidden focusable" href="#block-navigationprincipale">
    Aller au menu principal
   </a>
  </li>
  <li class="list-inline-item">
   <a class="skip-links__link visually-hidden focusable" href="#main-content">
    Aller au contenu principal
   </a>
  </li>
 </ul>
</div>



<ul class="list-inline m-0 p-0">
<li class="list-inline-item">
<a class="skip-links__link visually-hidden focusable" data-a11y-dialog-show="modal-dialog-access-config" data-target="#a11yModal" data-toggle="modal" href="#modal_access" role="button"> Open accessibility window </a>
</li>
<li class="list-inline-item">

In [59]:
for el in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em', 'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p'], string = True):
    print(el.text)

De la GPEC et au Workforce Planning : Les 5 évolutions à connaître
Une philosophie nouvelle
Une philosophie nouvelle
L’accélération de la transformation des métiers, couplée à un environnement économique incertain, conduit à réinventer l’approche GPEC pour qu’elle réponde à des questions opérationnelles d’anticipation des besoins en compétences et d’évolution des effectifs. Il s’agit de fournir aux équipes RH des outils de pilotage identiques à ceux déployés dans le reste de l’entreprise afin qu’elles puissent modéliser rapidement les impacts RH des décisions stratégiques ou des ruptures d’activité comme celles que nous avons vécus avec la crise sanitaire. L’exercice d’anticipation est fait régulièrement et donne lieu à des plans d’actions opérationnels suivis par les RRH et les managers grâce au outils digitaux.
La fin des « référentiels statiques »
La fin des « référentiels statiques »
Pour que cela fonctionne, les entreprises ne peuvent plus passer 2 à 3 ans à construire et déployer

In [56]:
from itertools import groupby

l = [list(j) for i, j in groupby(parents)]
idx = np.argmax(map(len, l))
print(list(set(l[idx]))[0].text.strip())

Historiquement, la GPEC est un exercice mené dans les entreprises pour répondre à des impératifs légaux de dialogue social. L’exercice, réalisé en moyenne tous les 3 ans, est souvent perçu comme théorique, chronophage et fastidieux.  Pourtant, depuis quelques mois, le terme de Workforce Planning revient sur le devant de la scène pour matérialiser la nouvelle dynamique de la GPEC. Voici les 5 évolutions à maîtriser pour comprendre l’évolution en cours :Une philosophie nouvelleL’accélération de la transformation des métiers, couplée à un environnement économique incertain, conduit à réinventer l’approche GPEC pour qu’elle réponde à des questions opérationnelles d’anticipation des besoins en compétences et d’évolution des effectifs. Il s’agit de fournir aux équipes RH des outils de pilotage identiques à ceux déployés dans le reste de l’entreprise afin qu’elles puissent modéliser rapidement les impacts RH des décisions stratégiques ou des ruptures d’activité comme celles que nous avons véc

In [37]:

print(url)
get_content(url)[1]

https://www.myrhline.com/actualite-rh/de-la-gpec-et-au-workforce-planning-les-5-evolutions-a-connaitre.html
[]


ValueError: attempt to get argmax of an empty sequence